# Filtro de Kalman
<span style="color: gray">dic 2019</span><br>
[*Alberto Ruiz*](http://dis.um.es/profesores/alberto)

La inferencia Bayesiana puede realizarse de forma analítica cuando las variables involucradas son [normales](https://en.wikipedia.org/wiki/Multivariate_normal_distribution) (gaussianas).

## Modelo gaussiano

Las operaciones de marginalización, condicionamiento y conjunción pueden calcularse de forma analítica cuando las variables son normales.

Son el fundamento del filtro de Kalman y de los [procesos gaussianos](https://en.wikipedia.org/wiki/Gaussian_process), entre otras muchas aplicaciones.

El [artículo de wikipedia](https://en.wikipedia.org/wiki/Kalman_filter) está bastante bien. Puede ser útil repasar los apartados C.5 y C.6 de mis [apuntes](http://dis.um.es/profesores/alberto/material/percep.pdf). 

### Expresiones analíticas

Si reordenamos las variables en dos grupos $(\boldsymbol x, \boldsymbol y)$ la densidad conjunta se puede expresar en función de las medias y matrices de covarianza de cada grupo y la covarianza cruzada:

$$ 
\newcommand{\mat}[1]{\boldsymbol{\mathtt #1}}
\newcommand{\T}{^\mathsf T}
\newcommand{\vec}[1]{\boldsymbol{#1}}
\newcommand{\I}{^{-1}}
p(\vec x, \vec y) \sim \mathcal N\left(\begin{bmatrix}\vec \mu_x \\ \vec \mu_y\end{bmatrix}, \begin{bmatrix}\Sigma_{xx} & \Sigma_{xy} \\ \Sigma_{yx} & \Sigma_{yy} \end{bmatrix}\right) = \mathcal N \left(\begin{bmatrix}\vec a \\ \vec b\end{bmatrix}, \begin{bmatrix}\mat A & \mat C\T \\ \mat C & \mat B \end{bmatrix}\right) $$


La densidad **marginal** de cualquier grupo se obtiene simplemente seleccionando las variables deseadas tanto en la media como en la matriz de covarianza. Por ejemplo:

$$p(\vec y) \sim  \mathcal N \left(\vec b, \mat B\right) $$


La densidad de un grupo de variables **condicionada** a la observación de otro grupo de variables es también gaussiana y se puede expresar de la siguiente forma:

$$
p(\vec y \mid \vec x) \sim \mathcal N \left(\vec b + \mat C \mat A\I (\vec x - \vec a)\; , \; \mat B - \mat C \mat A\I \mat C\T\right)
$$

(La media condicionada de esta gaussiana es la recta de regresión lineal).

En ocasiones estamos interesados realizar inferencia sobre unas variables $\vec x$ a partir de la observación de una cierta función de ellas: $\vec y = f(\vec x)$. Si $\vec x$ es gaussiana y la función $f$ es lineal podemos obtener fácilmente la densidad **conjunta** $p(\vec x,\vec y)$, que también es gaussiana, y realizar el condicionamiento como se acaba de explicar.

Concretamente, sea $p(\vec x) \sim \mathcal N (\vec \mu, \mat P)$ y $f(\vec x) = \mat H \vec x$ con ruido gaussiano aditivo de media $\vec o$ y covarianza $\mat R$. Esto significa que $p(\vec y| \vec x) \sim \mathcal N(\mat H \vec x + \vec o, \mat R)$. Entonces la densidad conjunta es:

$$
p(\vec x, \vec y) \sim \mathcal N\left(\begin{bmatrix}\vec \mu \\ \mat H \vec \mu + \vec o\end{bmatrix}, \begin{bmatrix}\mat P & \mat P \mat H\T \\ \mat H \mat P & \mat H \mat P \mat H\T + \mat R\end{bmatrix}\right)
$$


Y la densidad condicionada contraria $p(\vec x \mid \vec y)$ es:

$$p(\vec x \mid \vec y) \sim \mathcal N \left(\vec \mu + \mat K (\vec y - \mat H \vec \mu - \vec o) , (\mat I - \mat K \mat H )\mat P \right)$$

donde

$$ \mat K= \mat P \mat H\T (\mat H \mat P \mat H\T + \mat R)\I $$

Esta expresión está construida de manera que a partir de la observación $\vec y$
corregimos la información sobre $\vec x$ con una "ganancia de Kalman" $\mat K$ que depende
del balance entre la incertidumbre a priori $\mat P$, el ruido de la medida $\mat R$, y el modelo de medida $\mat H$.


Otra forma de verlo: la densidad conjunta se puede expresar de dos formas: modelo de medida $\times$ prior = posterior $\times$ evidencia

$$p(\vec y \mid \vec x) \; p(\vec x)  =  p(\vec x \mid \vec y) \;  p(\vec y) $$

$$\mathcal N (\vec y \mid \mat H \vec x + \vec o, \mat R) \;
\mathcal N (\vec x \mid \vec \mu, \mat P) =
\mathcal N (\vec x \mid \vec \eta_{\vec y}, \mat Q) \;
\mathcal N (\vec y \mid \mat H \vec \mu + \vec o, \mat H \mat P \mat H\T + \mat R)$$


La incertidumbre inicial sobre $\vec x$ era $\mat P$, que se reduce a $\mat Q$ tras la observación de $\vec y$:

$$ \mat Q\I = \mat P\I + \mat H\T \mat R\I \mat H $$

Y el estimador de $\vec x$ se actualiza de $\vec \mu$ a $\vec \eta _ {\vec y}$, que puede expresarse como una combinación ponderada de la observación y la información a priori:

$$\vec \eta _ {\vec y} = (\mat Q \mat H\T \mat R\I) (\vec y -\vec o) + (\mat Q \mat P\I) \vec \mu $$

La "evidencia" $p(\vec y)$ es la verosimilitud de la medida $\vec y$ teniendo en cuenta todos los posibles $\vec x$ (convolución de dos gaussianas). Juega un papel esencial en la selección de modelos.


### Experimentos

Para hacer experimentos usaremos una implementación de estas operaciones disponible en `umucv.prob`.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from   mpl_toolkits.mplot3d import Axes3D
%matplotlib inline

%pip install -q https://raw.githubusercontent.com/albertoruiz/jupyterlite/main/content/misc/umucv-0.3-py3-none-any.whl

from umucv.prob import G

En primer lugar veremos un ejemplo muy simple. Considera la siguiente gaussiana de dos componentes, cuya densidad se muestra como una superficie en 3D y como una elipse de incertidumbre.

In [ ]:
g = G([2,3], [[4,3],
              [3,4]])

fig = plt.figure(figsize=(10,5))
x = np.linspace(-3,7,50)
y = np.linspace(-2,8,50)
x1,x2 = np.meshgrid(x,y)
gxy = g.logprob()
z = np.array([[np.exp(gxy(np.array([x,y]))) for x in x] for y in y])

ax = fig.add_subplot(121, projection='3d')
ax.plot_surface(x1,x2,z, cmap='coolwarm', linewidth=0.5, rstride=2, cstride=2);
ax.view_init(elev=40,azim=90)
ax.set_xlabel('x'); ax.set_ylabel('y'); ax.set_title('p(x,y)'); ax.set_zticks([])

plt.subplot(1,2,2)
plt.plot(*g.ellipse().T);
plt.plot(*g.m,'.');
plt.xlabel('x'); plt.ylabel('y'); plt.title('elipse de incertidumbre'); plt.axis('equal');

En el siguiente diagrama se muestra la densidad conjunta $p(x,y)$, las dos densidades marginales, y la densidad de $x$ condicionada a un valor de y.

In [ ]:
plt.figure(figsize=(5,5))
plt.plot(*g.ellipse().T, label='$p(x,y)$');
px = g.marg([0]).logprob()
py = g.marg([1]).logprob()
plt.plot(x, [-3+10*np.exp(px(x)) for x in x], label='p(x)');
plt.plot([-4+10*np.exp(py(y)) for y in y], y, label='p(y)');
gx = g.cond([6]).logprob()
plt.plot(x, [-3+10*np.exp(gx(x)) for x in x], label='p(x|y=6)');
plt.plot([-4,7],[6,6],ls='dashed',color='gray');
plt.xlabel('x'); plt.ylabel('y'); plt.axis('equal');
plt.legend(loc=(1.04,0), fontsize=15);

La misma situación en 3D, donde se muestra el corte producido por la observación $y=5$ en la densidad conjunta, (verosimilitud o likelihood), y la probabilidad condicionada, que es simplemente la normalización del corte.

In [ ]:
fig = plt.figure(figsize=(6,5))
ax = fig.add_subplot(111, projection='3d')
ax.plot_surface(x1,x2,z, cmap='coolwarm', linewidth=0.5, rstride=1, cstride=1);
ax.view_init(elev=50,azim=60)
ax.set_xlabel('x'); ax.set_ylabel('y'); ax.set_title('p(x,y)'); ax.set_zticks([])


yobs = 5

z6 = [np.exp(gxy(np.array([x,yobs]))) for x in x]

ax.plot3D(x, yobs+x*0, z6, label="$p(x , y=5)$");
ax.plot3D(x, x*0+8, [1/5*np.exp(gx(x)) for x in x],label="$p(x\mid y=5)$");
plt.legend();

Veamos ahora la densidad conjunta de una variable $x\sim \mathcal N[0,2]$ y una función lineal de ella $y = 2x + 5 + \mathcal N[0,1]$:

In [ ]:
g = G([0],[[4]]).jointLinear([[2]], G([5], [[1]]))
                            
plt.figure(figsize=(5,5))
plt.plot(*g.ellipse().T); plt.axis('equal'); plt.grid(ls='dotted');
plt.xlabel('x'); plt.ylabel('y');
print(g.m)
print(g.c)

Calculamos la densidad condicionada a y=0 a partir de la densidad conjunta:

In [ ]:
# se condiciona los últimos elementos del vector
g = g.cond([0])
g.m, g.c

Y hacemos lo mismo con la fórmula directa que usa la ganancia K, sin pasar por la densidad conjunta:

In [ ]:
g = G([0],[[4]]).bayesGaussianLinearK([[2]], G([5],[[1]]), [0])
g.m, g.c

Veamos ahora un caso más interesante, donde $\vec x$ tiene dos componentes, que observamos con ruido.

In [ ]:
g = G([0,0] , [[4,3],
               [3,4]])

plt.figure(figsize=(5,5))
plt.plot(*g.ellipse().T); plt.axis('equal'); plt.grid(ls='dotted');

La densidad conjunta "estado"-"observación" es de dimensión 4.

In [ ]:
error = np.diag([0.4,0.1])
g.jointLinear(np.eye(2), G([0,0], error)).c

Dada una observación reducimos la incertidumbre:

In [ ]:
obs = [0,-3]
plt.figure(figsize=(5,5))
plt.axis('equal'); 
post = g.bayesGaussianLinear(np.eye(2), G([0,0], error),  obs )
plt.plot(*g.ellipse().T, label='prior'); 
plt.plot(*post.ellipse().T, label='posterior', color='blue');
plt.plot(*G(obs,  error).ellipse().T, label='likelihood');
plt.grid(ls='dotted'); plt.legend(loc=(1.04,0));

El siguiente ejemplo es la clave del filtro de Kalman: el estado tiene dos variables $(x,y)$, pero la observación es incompleta $z=x+y$ y ruidosa. También reduce la incertidumbre sobre el estado:

In [ ]:
plt.figure(figsize=(5,5))
post = g.bayesGaussianLinear([[1,1]], G([0], [[0.4]]),  [2] )
plt.plot(*g.ellipse().T, label='prior'); 
plt.plot(*post.ellipse().T, label='posterior', color='blue');
plt.grid(ls='dotted'); plt.legend(loc=(1.04,0));
plt.plot([-2,4],[4,-2],color='gray',ls='dashed');

## Implementación

Una implementación sencilla del filtro de Kalman y del UKF está disponible en el módulo `umucv.kalman`.

In [ ]:
%matplotlib inline

import numpy as np
import numpy.linalg as la 
import matplotlib.pyplot as plt
from matplotlib.patches import Ellipse

from umucv.kalman import kalman, ukf
import cv2
import umucv.htrans as ht

degree = np.pi/180

Por comodidad la incluímos aquí:

In [ ]:
def mikalman(mu,P,F,Q,B,u,z,H,R):
    # mu, P : estado actual y su incertidumbre
    # F, Q  : sistema dinámico y su ruido
    # B, u  : control model y la entrada
    # z     : observación
    # H, R  : modelo de observación y su ruido
    
    mup = F @ mu + B @ u;
    pp  = F @ P @ F.T + Q;

    zp = H @ mup

    # si no hay observación solo hacemos predicción 
    if z is None:
        return mup, pp, zp

    epsilon = z - zp

    k = pp @ H.T @ np.linalg.inv(H @ pp @ H.T +R)

    new_mu = mup + k @ epsilon;
    new_P  = (np.eye(len(P))-k @ H) @ pp;
    return new_mu, new_P, zp

## Ilustración 1-D

Vamos a resolver un problema sintético en el que un objeto se mueve en una única dimensión $x$ con una aceleración constante $a$. Desconocemos la velocidad inicial $v_0$ y solo observamos su posición a lo largo del tiempo, contaminada con ruido gaussiano de desviación $\sigma_r$.

In [ ]:
v0 = 0.5
a = -0.005
dt = 1
t = np.arange(0,100,dt)

sigmaR = 1
zp = v0*t + 1/2*a*t**2
zs = zp + sigmaR*np.random.randn(len(t));

plt.plot(t,zs);
plt.title("observaciones ruidosas de la posición"); plt.xlabel("t"); plt.ylabel("x");

El modelo del sistema es el siguiente:

$$
\begin{align*}
x_{k+1} &= x_k + \Delta t \; v_k \\
v_{k+1} &= v_k 
\end{align*}
$$

Suponiendo $\Delta t=1$, lo expresamos en forma matricial como transformaciones lineales del vector de estado:

In [ ]:
# modelo de evolución del sistema
f = np.array(
    [[1, dt],
     [0,  1]])

# control
B = np.array([[dt**2/2],[dt]])
u = np.array([a])

# el ruido del proceso se puede poner como incertidumbre en la aceleración
sigmaa = np.array([[abs(a/100)]])
s = B @ sigmaa**2 @ B.T

In [ ]:
#modelo de la observación
H = np.array([[1,0]])

#y su ruido
r = np.array([[sigmaR**2]])

In [ ]:
#el estado inicial
mu0 = np.array([5,0])

p0 = np.array(
    [[100000, 0],
     [0, 100000]])

Calculamos la estimación del vector de estado (posición y velocidad) para cada nueva observación:

In [ ]:
mu = mu0
p = p0

res = np.array([[mu[0],mu[1],np.sqrt(p[0,0]),np.sqrt(p[1,1])]])

for z in zs:
    mu,p,_ = kalman(mu,p,f,s,B,u,z,H,r)
    res = np.append(res,[[mu[0],mu[1],np.sqrt(p[0,0]),np.sqrt(p[1,1])]],axis=0)

print(res[:10])

In [ ]:
print(res[-5:])

In [ ]:
plt.plot(t,zs,t,zp);
plt.plot(t,res[1:,0],t,res[1:,0] + 2*res[1:,2],t,res[1:,0] - 2*res[1:,2],color='red');

In [ ]:
plt.plot(t,zs,t,zp,t,res[1:,0]);

Predicción sin observación: en un momento dado se pierden las medidas y la estimación se hace "a ciegas".

In [ ]:
mu = mu0
p = p0

res = np.array([[mu[0],mu[1],np.sqrt(p[0,0]),np.sqrt(p[1,1])]])

ran = 50

for z in zs[:ran]:
    mu,p,_ = kalman(mu,p,f,s,B,u,z,H,r)
    res = np.append(res,[[mu[0],mu[1],np.sqrt(p[0,0]),np.sqrt(p[1,1])]],axis=0)

for z in zs[ran:]:
    mu,p,_ = kalman(mu,p,f,s,B,u,None,H,r)
    res = np.append(res,[[mu[0],mu[1],np.sqrt(p[0,0]),np.sqrt(p[1,1])]],axis=0)

    
# extraemos las varianzas de la estimación de posición
# para dibujar la banda de incertidumbre
plt.plot(t[:ran],zs[:ran],t,zp);
plt.plot(t,res[1:,0],t,res[1:,0] + 2*res[1:,2],t,res[1:,0] - 2*res[1:,2],color='red');

Evolución de las elipses de incertidumbre.

In [ ]:
from umucv.prob import G

g0 = G(np.array([5,0]), 
       np.array([[100, 0],
                 [0, 100]]))

def showkalman2(g,z):
    g1 = G( f @ g.m + B @ u  ,    f @ g.c @ f.T + s );
    m,c,_ = kalman(g.m,g.c,f,s,B,u,z,H,r)
    g2 = G(m,c)
    plt.plot(*g1.ellipse().T,color='green');
    if z is not None:
        plt.plot(*g2.ellipse().T,color='blue');
    return g2

plt.figure(figsize=(10,5))
plt.plot(*g0.ellipse().T,color='orange');
g = g0
for z in [12,23,36,47,None,None]:
    g = showkalman2(g,z)
plt.axis('equal'); plt.xlabel('x'); plt.ylabel('v');

## Tiro parabólico 2D

[Tiro parabólico](http://hyperphysics.phy-astr.gsu.edu/hbase/traj.html#tra6).



TFG de Pablo Saura (UMU, 2017).

- [vídeo 1](https://www.youtube.com/watch?v=MxwVwCuBEDA)

- [vídeo 2](https://www.youtube.com/watch?v=YlPOTxYvt6U)

In [ ]:
x0 = np.array([0,0])
angle = 80*degree
v0 = 10*np.array([np.cos(angle), np.sin(angle)])

a = np.array([0, - 9.8])
t = np.arange(0,2.01,0.1)

Z = xp,yp = ht.col(x0)  +  ht.col(v0)* ht.row(t) + 1/2 * ht.col(a) * ht.row(t**2)

plt.figure(figsize=(8,6))
plt.plot(xp,yp,'.-',markersize=10);
plt.axis('equal'); plt.grid(); plt.xlabel('$x$ (m)'); plt.ylabel('$y$ (m)');
plt.title('trayectoria ideal ($\Delta t = 0.1s$)');

In [ ]:
fps = 25
dt  = 1/fps

F = np.array(
    [1, 0,  dt,  0,  
     0, 1,  0, dt,  
     0, 0,  1,  0,  
     0, 0,  0,  1 ]).reshape(4,4)


B = np.array(
         [dt**2/2, 0,       
          0,       dt**2/2, 
          dt,      0,       
          0,       dt      ]).reshape(4,2)


H = np.array(
    [1,0,0,0,
     0,1,0,0]).reshape(2,4)

In [ ]:
def evol(x,u):
    return F@x + B@u

In [ ]:
s0 = np.hstack([x0,v0])

r = [s0]
s = s0
for k in range(round(2/dt)):
    s = evol(s,a)
    r.append(s)
    #print(s)
r = np.array(r)

In [ ]:
xt,yt,*vs = r.T

In [ ]:
plt.figure(figsize=(8,6))
plt.plot(xt,yt,'.-',markersize=10); plt.axis('equal'); plt.grid();
plt.xlabel('$x$ (m)'); plt.ylabel('$y$ (m)');
plt.title('comprobación del modelo ($\Delta t = (1/25) s$)');

In [ ]:
x0 = np.array([0,0])
angle = 80*degree
v0 = 10*np.array([np.cos(angle), np.sin(angle)])

a = np.array([0, - 9.8])
t = np.arange(0,2.01,dt)

noise = 0.05

# trayectoria verdadera (desconocida en realidad)
Zt = xt,yt = ht.col(x0)  +  ht.col(v0)* ht.row(t) + 1/2 * ht.col(a) * ht.row(t**2)

# trayectoria observada
Z  = xo,yo = Zt + noise*np.random.randn(2,len(t))

plt.figure(figsize=(8,6))
plt.plot(xo,yo,'.-',markersize=10);
plt.axis('equal'); plt.grid(); plt.xlabel('$x$ (m)'); plt.ylabel('$y$ (m)');
plt.title('trayectoria observada ($\Delta t = (1/25) s$)');

In [ ]:
# estado que Kalman va actualizando. Este es el valor inicial

             # x, y, vx, vy
mu = np.array([0,0,0,0])
            # sus incertidumbres 
P  = np.diag([100,100,100,100])**2
#res = [(mu,P,mu)]
res=[]
N = 15  # para tomar un tramo inicial y ver qué pasa si luego se pierde la observación

sigmaM = 0.001   # ruido del modelo
sigmaZ = 3*noise  # debería ser igual al ruido de media del proceso de imagen. 10 pixels pje.

Q = sigmaM**2 * np.eye(4)
R = sigmaZ**2 * np.eye(2)

In [ ]:
                                    # z es la medida del centro de la pelota observada
                                    # mu es la estimación filtrada actualizada
for z in Z.T[1:N]:
    mu,P,pred = kalman(mu,P,F,Q,B,a,z,H,R)
    res += [(mu,P,pred)]

for _ in range(50-N):
    mu,P,pred = kalman(mu,P,F,Q,B,a,None,H,R)  # aquí solo continuamos la predicción
    res += [(mu,P,pred)]
    

res = np.array(res)

In [ ]:
xe = [mu[0] for mu,_,_ in res]             # coordenada x estimada
xu = [2*np.sqrt(P[0,0]) for _,P,_ in res]  # su incertidumbre

ye = [mu[1] for mu,_,_ in res]             # lo mismo para y
yu = [2*np.sqrt(P[1,1]) for _,P,_ in res]

In [ ]:
fig,ax = plt.subplots(figsize=(8,8))

for k in range(len(xe)):
    ax.add_patch(Ellipse(xy=(xe[k],ye[k]), width=xu[k], height=yu[k], angle = 0, alpha=0.2))

plt.plot(xe,ye,lw=1)
plt.plot(xo[1:N],yo[1:N])#,xe,ze);
plt.plot(xt,yt,lw=0.5,color='gray'); plt.grid()
plt.axis('equal');

In [ ]:
from matplotlib import animation, rc
from IPython.display import HTML
rc('animation', html='html5')

In [ ]:
fig, ax = plt.subplots(figsize=(8,8))
plt.close();
ax.set_xlim(( -1, 4))
ax.set_ylim(( 0, 5))

line1, = ax.plot([], [], lw=1)
line2, = ax.plot([],[])
line3, = ax.plot([],[],'.',markersize=15)
line4, = ax.plot([],[])
line5, = ax.plot([],[])
line6, = ax.plot(xt,yt,lw=0.5,color='gray')

mu0 = np.array([0,0,0,0])
            # sus incertidumbres 
P0  = np.diag([10,10,10,10])**2
#res = [(mu,P,mu)]

def animate(i):
    global mu,P
    N = i
    res=[]
    mu = mu0
    P  = P0
    for z in Z.T[1:N]:
        mu,P,pred = kalman(mu,P,F,Q,B,a,z,H,R)
        #print(mu)
        res += [(mu,P,pred)]

    for _ in range(len(Z.T)-N):
        mu,P,pred = kalman(mu,P,F,Q,B,a,None,H,R)  # aquí solo continuamos la predicción
        res += [(mu,P,pred)]
    

    res = np.array(res)
    
    xe = np.array([mu[0] for mu,_,_ in res])
    xu = np.array([2*np.sqrt(P[0,0]) for _,P,_ in res])

    ye = np.array([mu[1] for mu,_,_ in res])
    yu = np.array([2*np.sqrt(P[1,1]) for _,P,_ in res])
    
    line1.set_data(xe,ye)
    line2.set_data(xo[1:N],yo[1:N])
    line3.set_data(xo[max(0,N-1)],yo[max(0,N-1)])  
    line4.set_data(xe+xu,ye)
    line5.set_data(xe-xu,ye)
    
    return ()

animation.FuncAnimation(fig, animate, init_func=lambda:[], frames=50, interval=4*1000/25, blit=True)